In [1]:
import os

In [4]:
eletricity_files = os.listdir('dataset/eletricity/')
eletricity_files

['2020年度每日尖峰備轉容量率.csv', '2021年度每日尖峰備轉容量率.csv']

In [5]:
weather_files = os.listdir('dataset/weather/')
weather_files

['weather_day.csv', 'weather_forcast.csv']

In [6]:
import pandas
import numpy as np

# Electicity Data

In [8]:
edf1 = pandas.read_csv(os.path.join('dataset/eletricity', eletricity_files[0]))

edf1

,日期,備轉容量(萬瓩),備轉容量率(%)
0,2020/01/01,279.0,11.01
1,2020/01/02,308.0,10.91
2,2020/01/03,338.3,11.96
3,2020/01/04,363.6,14.21
4,2020/01/05,269.5,10.88
...,...,...,...
345,2020/12/11,312.9,10.46
346,2020/12/12,328.2,12.31
347,2020/12/13,283.0,11.12
348,2020/12/14,302.1,10.20


In [9]:
edf2 = pandas.read_csv(os.path.join('dataset/eletricity', eletricity_files[1]))

edf2

,日期,備轉容量(萬瓩),備轉容量率(%)
0,2021/01/01,276.7,10.76
1,2021/01/02,293.6,11.52
2,2021/01/03,303.7,11.92
3,2021/01/04,303.2,10.20
4,2021/01/05,306.7,10.21
...,...,...,...
75,2021/03/17,318.3,10.36
76,2021/03/18,321.4,10.46
77,2021/03/19,313.7,10.16
78,2021/03/20,283.9,10.23


In [63]:
eletricities = np.concatenate([edf1['備轉容量(萬瓩)']*10, edf2['備轉容量(萬瓩)']*10])
len(eletricities[-357:])

357

# Weather Data

In [10]:
wdf1 = pandas.read_csv(os.path.join('dataset/weather', weather_files[0]))

wdf1

,Unnamed: 0,日期,locationName,當日最高溫度(°C),當日最低溫度(°C),當日平均溫度(°C)
0,0,2020-03-18,"BANQIAO,板橋",24.1,17.0,20.3
1,1,2020-03-19,"BANQIAO,板橋",21.9,18.3,19.7
2,2,2020-03-20,"BANQIAO,板橋",25.2,18.7,21.7
3,3,2020-03-21,"BANQIAO,板橋",30.1,19.4,23.9
4,4,2020-03-22,"BANQIAO,板橋",32.0,20.8,25.2
...,...,...,...,...,...,...
10947,10947,2021-03-14,"TianZhong,田中",25.9,16.3,20.3
10948,10948,2021-03-15,"TianZhong,田中",26.4,16.7,21.1
10949,10949,2021-03-16,"TianZhong,田中",29.5,18.3,23.8
10950,10950,2021-03-17,"TianZhong,田中",28.4,19.4,23.1


In [35]:
# eletricity data only to 2020-12-15
dates2020 = sorted(list(set(wdf1['日期'])))[:273]
print(dates2020[0], dates2020[-1])

2020-03-18 2020-12-15


In [36]:
dates2021 = sorted(list(set(wdf1['日期'])))[289:]
print(dates2021[0], dates2021[-1])

2021-01-01 2021-03-18


In [40]:
avgs_temperature = []
for i in dates2020:
    avgs_temperature.append(np.average(wdf1[wdf1['日期'] == i]['當日平均溫度(°C)']))

for i in dates2021:
    avgs_temperature.append(np.average(wdf1[wdf1['日期'] == i]['當日平均溫度(°C)']))


In [42]:
len(avgs_temperature)

350

In [43]:
# read forcast weather data
wdf2 = pandas.read_csv(os.path.join('dataset/weather', weather_files[1]))

wdf2

,Unnamed: 0,北部地區,中部地區,南部地區
0,2021-03-20,12.5,14.0,16.0
1,2021-03-21,8.0,12.0,15.0
2,2021-03-22,4.5,9.0,11.0
3,2021-03-23,8.0,10.5,12.0
4,2021-03-24,9.5,11.5,13.0
5,2021-03-25,10.0,12.5,14.0
6,2021-03-26,11.0,12.5,14.5


In [49]:
for i in range(len(wdf2)):
    avgs_temperature.append(np.average(wdf2.iloc[i][1:4]))

In [50]:
len(avgs_temperature)

357

# dataset

In [59]:
from sklearn.preprocessing import normalize
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset


In [99]:

class Dataset(BaseDataset):
    def __init__(
        self,
        train_data_electricity,
        train_data_weather,
    ):
        #prepare the training data and label
        self.train_label = np.array(train_data_electricity, dtype=np.float32)
        self.train_data_electricity = np.array(train_data_electricity,  dtype=np.float32)
        self.train_data_weather = np.array(train_data_weather,  dtype=np.float32)
    
    def __getitem__(self, i):
        x = np.concatenate([self.train_data_electricity[i:30+i],  self.train_data_weather[i:30+i]])
        y = self.train_label[30+i:37+i]
        
        return x, y
    
    def __len__(self):
        return len(self.train_data_electricity) - 36

In [107]:
dataset = Dataset(train_data_electricity=eletricities[-len(avgs_temperature):-30], train_data_weather=avgs_temperature[:-30])

In [108]:
len(dataset)

291

In [109]:
x,y = dataset[1]

In [110]:
print(len(x), len(y))

60 7


In [111]:
x,y = dataset[len(dataset)-1]

In [112]:
y

array([3024., 2911., 2645., 3002., 2824., 2907., 2885.], dtype=float32)

# Model

In [113]:
import torch
import torch.nn as nn

In [114]:
class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel,self).__init__()
        
        self.fc0 = nn.Linear(60, 128)
        self.fc1 = nn.Linear(128, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc = nn.Linear(128, 7)
        self.flatten = nn.Flatten()
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.fc0(x)
        x = self.relu(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc(x)
        
        return x

In [115]:
linear = LinearModel()

In [116]:
x,y = dataset[0]
x = torch.tensor(x, dtype=torch.float).unsqueeze(dim=0)
y = torch.tensor(y, dtype=torch.float).unsqueeze(dim=0)

In [118]:
linear.cuda()

LinearModel(
  (fc0): Linear(in_features=60, out_features=128, bias=True)
  (fc1): Linear(in_features=128, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc): Linear(in_features=128, out_features=7, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (relu): ReLU()
)

In [119]:
linear(x.cuda())

tensor([[-53.8312, -64.6541, -56.0945,  23.4642, -99.5560, -24.3962,  45.2307]],
       device='cuda:0', grad_fn=<AddmmBackward>)

In [120]:
# Training